## Initialize Spark Environment

In [1]:
import findspark
findspark.init('/home/shahayush954/spark-3.4.1-bin-hadoop3')

## Creating Spark Session

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ml').getOrCreate()

23/08/14 23:42:28 WARN Utils: Your hostname, ubuntu-22 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/08/14 23:42:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/14 23:42:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Importing ML Libraries and loading data

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')
training.show()

23/08/14 23:45:42 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

## Training

In [5]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [6]:
lrModel = lr.fit(training)

23/08/14 23:48:52 WARN Instrumentation: [42575046] regParam is zero, which might cause numerical instability and overfitting.


## Interpreting the Training Results

In [7]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [8]:
lrModel.intercept

0.14228558260358093

In [9]:
training_summary = lrModel.summary

In [10]:
training_summary.r2

0.027839179518600154

In [11]:
training_summary.rootMeanSquaredError

10.16309157133015

## Spiliting Data into Training and Testing and Running again

In [14]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

23/08/14 23:55:23 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [17]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [18]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
| -16.08565904102149|(10,[0,1,2,3,4,5,...|
|-15.951512565794573|(10,[0,1,2,3,4,5,...|
+----------

In [19]:
correct_model = lr.fit(train_data)

23/08/14 23:58:34 WARN Instrumentation: [588beddf] regParam is zero, which might cause numerical instability and overfitting.


In [20]:
test_results = correct_model.evaluate(test_data)

In [21]:
test_results.rootMeanSquaredError

10.980210638702147

## Testing the Model

In [22]:
unlabelled_data = test_data.select('features')

In [23]:
unlabelled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [24]:
predictions = correct_model.transform(unlabelled_data)

In [26]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -1.0182589973283096|
|(10,[0,1,2,3,4,5,...| 0.17395816793200625|
|(10,[0,1,2,3,4,5,...|0.017338596368040166|
|(10,[0,1,2,3,4,5,...|  1.0497756670674403|
|(10,[0,1,2,3,4,5,...|  -0.979253800504582|
|(10,[0,1,2,3,4,5,...|  1.0631430351879945|
|(10,[0,1,2,3,4,5,...| -3.0907327937757896|
|(10,[0,1,2,3,4,5,...|  -1.264881271517742|
|(10,[0,1,2,3,4,5,...| -0.3728095710767464|
|(10,[0,1,2,3,4,5,...|    4.10703339620169|
|(10,[0,1,2,3,4,5,...|  1.3290855101067196|
|(10,[0,1,2,3,4,5,...|    1.48804339787339|
|(10,[0,1,2,3,4,5,...|   2.998398275013778|
|(10,[0,1,2,3,4,5,...| -3.8409757779448994|
|(10,[0,1,2,3,4,5,...|   4.336572180515074|
|(10,[0,1,2,3,4,5,...| -0.3746375531578251|
|(10,[0,1,2,3,4,5,...|   4.961708862885091|
|(10,[0,1,2,3,4,5,...|  -1.491153730469879|
|(10,[0,1,2,3,4,5,...|-0.13445990080025466|
|(10,[0,1,2,3,4,5,...|  -5.10377